# Notebook for Final Project: "Which Minneapolis Community is Closest to the Most Green Spaces"

In [4]:
import requests
import pandas as pd
import numpy as np
!conda install -c conda-forge geopy --yes 
from geopy.geocoders import Nominatim
from IPython.display import Image 
from IPython.core.display import HTML 
from pandas.io.json import json_normalize
!conda install -c conda-forge folium=0.5.0 --yes
import folium
import json

Solving environment: done

## Package Plan ##

  environment location: /opt/conda/envs/Python36

  added / updated specs: 
    - geopy


The following packages will be downloaded:

    package                    |            build
    ---------------------------|-----------------
    geopy-1.20.0               |             py_0          57 KB  conda-forge
    openssl-1.1.1d             |       h516909a_0         2.1 MB  conda-forge
    ca-certificates-2019.11.28 |       hecc5488_0         145 KB  conda-forge
    certifi-2019.11.28         |           py36_0         149 KB  conda-forge
    geographiclib-1.50         |             py_0          34 KB  conda-forge
    ------------------------------------------------------------
                                           Total:         2.5 MB

The following NEW packages will be INSTALLED:

    geographiclib:   1.50-py_0         conda-forge
    geopy:           1.20.0-py_0       conda-forge

The following packages will be UPDATED:

    ca-

### Creating dataframe from Communities.csv file

In [6]:
#import custom csv data
import types
from botocore.client import Config
import ibm_boto3

def __iter__(self): return 0

client_88eae2a4e9ac4633b409918259864261 = ibm_boto3.client(service_name='s3',
    ibm_api_key_id='xRWGLQd9YK29IctoIhRJkVi4ESWO86Mo4QIv0AQ0VkV0',
    ibm_auth_endpoint="https://iam.ng.bluemix.net/oidc/token",
    config=Config(signature_version='oauth'),
    endpoint_url='https://s3-api.us-geo.objectstorage.service.networklayer.com')

body = client_88eae2a4e9ac4633b409918259864261.get_object(Bucket='capstoneproject-donotdelete-pr-nr1mfkdz7lbpjc',Key='Communities.csv')['Body']
if not hasattr(body, "__iter__"): body.__iter__ = types.MethodType( __iter__, body )

communities = pd.read_csv(body)


## Foursquare Code

### Foursquare Credentials

In [7]:
#create foursquare credentials
CLIENT_ID = 'AQGGTBMBHZPH5I5OBYLDA23QKAHTJ300OIGO1MGSDPGF1RTB'
CLIENT_SECRET = 'X3NPOD4JKTPBXLM0O5RIOT3RYFZSSAYZDLQDLQWDFKDF3IYJ' 
VERSION = '20180604'
LIMIT = 100
radius= 8000
print('Your credentails:')
print('CLIENT_ID: ' + CLIENT_ID)
print('CLIENT_SECRET:' + CLIENT_SECRET)

Your credentails:
CLIENT_ID: AQGGTBMBHZPH5I5OBYLDA23QKAHTJ300OIGO1MGSDPGF1RTB
CLIENT_SECRET:X3NPOD4JKTPBXLM0O5RIOT3RYFZSSAYZDLQDLQWDFKDF3IYJ


### Retrieve coordinates of Minneapolis

In [8]:
#get latitude and longitude of Minneapolis
address = 'Minneapolis, Minnesota'

geolocator = Nominatim(user_agent="foursquare_agent")
location = geolocator.geocode(address)
latitude = location.latitude
longitude = location.longitude
print(latitude, longitude)

44.9706705 -93.268217735811


### Retrieve categories of different green spaces

In [9]:
#create category IDs for parks and lakes
category_ID = "4bf58dd8d48988d163941735"
category_ID2 = '4bf58dd8d48988d161941735'
category_ID3 = '4bf58dd8d48988d15f941735'
category_ID4 = '52e81612bcbc57f1066b7a23'
url = 'https://api.foursquare.com/v2/venues/search?client_id={}&client_secret={}&ll={},{}&v={}&categoryId={}&radius={}&limit={}'.format(CLIENT_ID, CLIENT_SECRET, latitude, longitude, VERSION, category_ID, radius, LIMIT)
url2 = 'https://api.foursquare.com/v2/venues/search?client_id={}&client_secret={}&ll={},{}&v={}&categoryId={}&radius={}&limit={}'.format(CLIENT_ID, CLIENT_SECRET, latitude, longitude, VERSION, category_ID2, radius, LIMIT)
url3 = 'https://api.foursquare.com/v2/venues/search?client_id={}&client_secret={}&ll={},{}&v={}&categoryId={}&radius={}&limit={}'.format(CLIENT_ID, CLIENT_SECRET, latitude, longitude, VERSION, category_ID3, radius, LIMIT)
url4 = 'https://api.foursquare.com/v2/venues/search?client_id={}&client_secret={}&ll={},{}&v={}&categoryId={}&radius={}&limit={}'.format(CLIENT_ID, CLIENT_SECRET, latitude, longitude, VERSION, category_ID4, radius, LIMIT)

### Retrieve results and create datasets

In [37]:
#get results of requests
results = requests.get(url).json()
results2 = requests.get(url2).json()
results3 = requests.get(url3).json()
results4 = requests.get(url4).json()

In [11]:
#create venues lists
venues = results['response']['venues']
venues2 = results2['response']['venues']
venues3 = results3['response']['venues']
venues4 = results4['response']['venues']

#put parks and lakes into 2 dataframes
parks = json_normalize(venues)
lakes = json_normalize(venues2)
fields = json_normalize(venues3)
forests = json_normalize(venues4)

In [12]:
# function that extracts the category of the venue
def get_category_type(row):
    try:
        categories_list = row['categories']
    except:
        categories_list = row['venue.categories']
        
    if len(categories_list) == 0:
        return None
    else:
        return categories_list[0]['name']


# keep only columns that include venue name, and anything that is associated with location
filtered_columns = ['name', 'categories'] + [col for col in parks.columns if col.startswith('location.')] + ['id']
parks_filtered = parks.loc[:, filtered_columns]

# filter the category for each row
parks_filtered['categories'] = parks_filtered.apply(get_category_type, axis=1)

# clean column names by keeping only last term
parks_filtered.columns = [column.split('.')[-1] for column in parks_filtered.columns]



filtered_columns = ['name', 'categories'] + [col for col in lakes.columns if col.startswith('location.')] + ['id']
lakes_filtered = lakes.loc[:, filtered_columns]

filtered_columns = ['name', 'categories'] + [col for col in fields.columns if col.startswith('location.')] + ['id']
fields_filtered = fields.loc[:, filtered_columns]

fields_filtered['categories'] = fields_filtered.apply(get_category_type, axis=1)

fields_filtered.columns = [column.split('.')[-1] for column in fields_filtered.columns]



filtered_columns = ['name', 'categories'] + [col for col in forests.columns if col.startswith('location.')] + ['id']
forests_filtered = forests.loc[:, filtered_columns]

forests_filtered['categories'] = forests_filtered.apply(get_category_type, axis=1)

forests_filtered.columns = [column.split('.')[-1] for column in forests_filtered.columns]

In [22]:
#remove duplicates
parks_filtered = parks_filtered.drop_duplicates(subset='name', keep='first')
lakes_filtered = lakes_filtered.drop_duplicates(subset='name', keep='first')
fields_filtered = fields_filtered.drop_duplicates(subset='name', keep='first')
forests_filtered = forests_filtered.drop_duplicates(subset='name', keep='first')

#rename location.lat and location.lng to lat and lng in lakes_filtered
lakes_filtered1 = lakes_filtered.rename(columns = {'location.lat': 'lat', 'location.lng': 'lng'})

,name,categories,location.address,location.cc,location.city,location.country,location.crossStreet,location.distance,location.formattedAddress,location.labeledLatLngs,lat,lng,location.neighborhood,location.postalCode,location.state,id
0,Loring pond,"[{'id': '4bf58dd8d48988d161941735', 'name': 'L...",NaN,US,Minneapolis,United States,NaN,1295,"[Minneapolis, MN, United States]","[{'label': 'display', 'lat': 44.969086, 'lng':...",44.969086,-93.284520,NaN,NaN,MN,575dafbb498e67516b50904e
1,Lake of the Isles,"[{'id': '4bf58dd8d48988d161941735', 'name': 'L...",Lake of the Isles Pkwy,US,Minneapolis,United States,btw Lake St & Franklin Ave,3493,[Lake of the Isles Pkwy (btw Lake St & Frankli...,"[{'label': 'display', 'lat': 44.955077, 'lng':...",44.955077,-93.306708,NaN,55405,MN,4a132038f964a520bc771fe3
2,Bde Maka Ska,"[{'id': '4bf58dd8d48988d161941735', 'name': 'L...",3000 E Calhoun Pkwy,US,Minneapolis,United States,btwn Lake St. & 38th St. W.,4621,[3000 E Calhoun Pkwy (btwn Lake St. & 38th St....,NaN,44.943113,-93.312097,West Calhoun,55408,MN,44784757f964a520c5331fe3
3,Cedar Lake,"[{'id': '4bf58dd8d48988d161941735', 'name': 'L...",Cedar Lake Pkwy.,US,Minneapolis,United States,btwn Cedar Lake Rd. S. & Cedar Lake Ave.,4304,[Cedar Lake Pkwy. (btwn Cedar Lake Rd. S. & Ce...,"[{'label': 'display', 'lat': 44.96054676111220...",44.960547,-93.320961,NaN,55416,MN,4a68dc31f964a52024cb1fe3
4,Birdtown,"[{'id': '4bf58dd8d48988d161941735', 'name': 'L...",France Ave,US,NaN,United States,Shoreline Drive,8902,"[France Ave (Shoreline Drive), Minnesota, Unit...","[{'label': 'display', 'lat': 45.0323633713234,...",45.032363,-93.340185,NaN,NaN,Minnesota,4e1cef9152b1944a8284fde9
5,Blackstone Park,"[{'id': '4bf58dd8d48988d161941735', 'name': 'L...",NaN,US,Minneapolis,United States,NaN,7004,"[Minneapolis, MN 55416, United States]","[{'label': 'display', 'lat': 44.9677854239442,...",44.967785,-93.357062,NaN,55416,MN,4c155265a5eb76b0bb2dc3b7
6,Thomas Beach,"[{'id': '4bf58dd8d48988d1e2941735', 'name': 'B...",3779 W Calhoun Pkwy,US,Minneapolis,United States,at Thomas Ave S,5403,"[3779 W Calhoun Pkwy (at Thomas Ave S), Minnea...","[{'label': 'display', 'lat': 44.93574084901812...",44.935741,-93.315839,NaN,55408,MN,4c191eeafe5a76b0a2190315
7,Silverwood Boathouse,"[{'id': '4bf58dd8d48988d161941735', 'name': 'L...",2500 County Road E W,US,Saint Anthony,United States,NaN,9054,"[2500 County Road E W, Saint Anthony, MN 55421...","[{'label': 'display', 'lat': 45.04614669011742...",45.046147,-93.225329,NaN,55421,MN,4fb52a19e4b0eb716639a99f
8,Grazier Park,"[{'id': '4bf58dd8d48988d163941735', 'name': 'P...",45th Ave N,US,Robbinsdale,United States,Perry Ave N,9522,"[45th Ave N (Perry Ave N), Robbinsdale, MN 554...","[{'label': 'display', 'lat': 45.03782324653506...",45.037823,-93.343170,NaN,55422,MN,4bcb6c983740b713647c6265
9,Powderhorn Lake,"[{'id': '4bf58dd8d48988d161941735', 'name': 'L...",3400 15th Ave S,US,Minneapolis,United States,btwn E. 35th St. & E. 32nd St.,3369,[3400 15th Ave S (btwn E. 35th St. & E. 32nd S...,"[{'label': 'display', 'lat': 44.9414770897418,...",44.941477,-93.256941,NaN,55407,MN,4bf866cb4a67c928476025cf


## Mapping data with Folium

In [36]:
map_minneapolis = folium.Map(location=[latitude, longitude], zoom_start=12, tiles = 'Stamen Terrain')

#add markers for parks
for lat, lng, name in zip(parks_filtered.lat, parks_filtered.lng, parks_filtered.name):
    label = '{}'.format(name)
    label = folium.Popup(label,parse_html=True)
    folium.features.CircleMarker(
        [lat, lng],
        radius=5,
        color='green',
        popup=label,
        fill = True,
        fill_color='green',
        fill_opacity=0.6
    ).add_to(map_minneapolis)
    
#add markers for lakes
for lat, lng, name in zip(lakes_filtered1.lat, lakes_filtered1.lng, lakes_filtered1.name):
    label = '{}'.format(name)
    label = folium.Popup(label,parse_html=True)
    folium.features.CircleMarker(
        [lat, lng],
        radius=5,
        color='green',
        popup=label,
        fill = True,
        fill_color='green',
        fill_opacity=0.6
    ).add_to(map_minneapolis)
    
#add markers for fields
for lat, lng, name in zip(fields_filtered.lat, fields_filtered.lng, fields_filtered.name):
    label = '{}'.format(name)
    label = folium.Popup(label,parse_html=True)
    folium.features.CircleMarker(
        [lat, lng],
        radius=5,
        color='green',
        popup=label,
        fill = True,
        fill_color='green',
        fill_opacity=0.6
    ).add_to(map_minneapolis)
    
#add markers for forests
for lat, lng, name in zip(forests_filtered.lat, forests_filtered.lng, forests_filtered.name):
    label = '{}'.format(name)
    label = folium.Popup(label,parse_html=True)
    folium.features.CircleMarker(
        [lat, lng],
        radius=5,
        color='green',
        popup=label,
        fill = True,
        fill_color='green',
        fill_opacity=0.6
    ).add_to(map_minneapolis)
    
#add markers for communities
for lat, long, Community in zip(communities['lat'], communities['long'], communities['Community']):
    label = '{}'.format(Community)
    label = folium.Popup(label, parse_html=True)
    folium.features.CircleMarker(
        [lat, long],
        radius=80,
        popup=label,
        color='red',
        fill= True,
        fill_color='red',
        fill_opacity=0.1,
        parse_html=False).add_to(map_minneapolis)    

map_minneapolis